In [1]:
# Variantas:  vieno įrašo detalus testavimas 

# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šis variantas pritaikytas npy formato zive įrašams, kuriems pakeistas, lyginant su 
# originaliais įrašais, failo vardas iš `file_name` į `SubjCode`, pridedant `userNr`
# prie `file_name`. 
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, funkcijos 
# paimamos iš aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1, testuojami duomenys
# iš db_folder įrašų saugyklos, jame yra ir failas all_beats_attr. 

# Testavimui imami įrašai iš sąrašo SubjCodes, kuris arba paimamas if failo info_create.json,
# arba iš mokymo, validavimo, testavimo sarašų, pvz. train_subjcode_lst.csv. Visiems įrašams iš šių
# sąrašų egzistuoja informacija apie pūpsnius faile all_beats_attr.
 
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota
 
import pandas as pd
import numpy as np
import time
import sys, json
from pathlib import Path
from icecream import ic

# from zive_util_vu import cm2df, show_confusion_matrix 
# from zive_util_vu import create_dir, create_subdir, get_rev_dictionary
# from zive_util_vu import runtime, split_SubjCode
# from zive_util_vu import get_freq_unique_values

# from zive_util_vu import get_beat_attributes
# from zive_util_vu import get_userId, read_rec, get_filename 
# from zive_util_vu import confusion_matrix_modified, zive_read_df_rpeaks

# from zive_cnn_fda_vu_v1 import predict_cnn_fda_vu_v1, zive_read_file_1ch
# Pastaba: zive_read_file_1ch importuoju iš zive_cnn_fda_vu_v1, nors ji yra ir zive_util_vu.py
# tam, kad atskirti funkcijas, kurios importuojamos skripte zive analysis, nuo tų funkcijų,
# kurios reikalingos tik zive_accuracy_cnn_vu_v1 ir v2. 


print("Skriptas Zive klasifikavimo rezultatų failo analizei")

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_VU'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3}  

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)

res_dir = Path(db_path, 'test_results', 'test')
# res_dir = '/home/test_results/test'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su EKG įrašais: ", rec_dir)
print("Aplankas su resultatais: ", res_dir)
print("Pūpsnių atributų failas:", all_beats_attr_fname)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))

print("\n")


Skriptas Zive klasifikavimo rezultatų failo analizei
OS in my system :  linux

Bendras duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE/VU
Zive duomenų aplankas:  DUOM_VU
Aplankas su EKG įrašais:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy
Aplankas su resultatais:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/test_results/test
Pūpsnių atributų failas: all_beats_attr_z.csv
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3




In [6]:
def zive_read_df_rpeaks(db_path, recordingId):
    file_path = Path(db_path, recordingId + '.json')
    with open(file_path,'r') as f:
        data = json.loads(f.read())
    df_rpeaks = pd.json_normalize(data, record_path =['rpeaks'])
    return df_rpeaks

def zive_read_df_data(file_path, name):
    df_data = pd.DataFrame()
    path = Path(file_path)
    # https://www.askpython.com/python-modules/check-if-file-exists-in-python
    if (path.exists()):
        # with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
        with open(file_path,'r') as f:
            data = json.loads(f.read())
        df_data = pd.json_normalize(data, record_path =[name])
    return df_data

def get_filename(rec_dir, SubjCode):
    userNr, recNr = split_SubjCode(SubjCode)
    if (userNr < 1000):
        return str(SubjCode)

    # Patikriname, ar df_transl egzistuoja. 
    file = Path(rec_dir, 'df_transl.csv')
    if (file.exists()):
        # Nuskaitome vardų žodyną iš rec_dir aplanko
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl = pd.read_csv(file_path, index_col=0)
#       print(df_transl) 
         # Panaudodami df masyvą df_transl su įrašų numeriais iš įrašų eilės numerių gauname ZIVE numerius
        row = df_transl.loc[(df_transl['userNr'] == userNr) & (df_transl['recordingNr'] == recNr)]
        if row.empty:
            print("Klaida!")
            return None
        else:
            return row['file_name'].values[0]  # naudojame values[0], nes gauname series elementą
    else:
        print("df_transl neegzistuoja")

def split_SubjCode(SubjCode):
    """
    Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
    
    zive atveju: SubjCode = int(str(userNr) + str(registrationNr)), kur userNr >= 1000,
    pvz. SubjCode = 10001
    mit2zive atveju: SubjCode = userNr,  kur userNr < 1000,
    pvz. SubjCode = 101
    https://www.adamsmith.haus/python/answers/how-to-get-the-part-of-a-string-before-a-specific-character-in-python
    Parameters
    ------------
        SubjCode: int
    Return
    -----------
        userNr: int
        recordingNr: int
    """   
    if (SubjCode < 1000):
        userNr = SubjCode
        recordingNr = 0   
        return userNr, recordingNr
    else:        
        str_code = str(SubjCode) 
        chars = list(str_code)
        str1 =""
        userNr = int(str1.join(chars[:4]))
        str2 =""
        recordingNr = int(str2.join(chars[4:]))
        return userNr, recordingNr
 


# file_names = ['1625402.027']
# SubjCodes = [10021, 10022, 10083, 10091] #Testavimui, iš mokymo imties
SubjCodes = [10021] #Testavimui, iš mokymo imties
print("Klasifikuojamų įrašų sąrašas:", SubjCodes)

for SubjCode in SubjCodes:
    
# I-a dalis: nuskaitome ir analizuojame EKG įrašo anotacijas

    
    # Surandame ir išvedame įrašo atributus
    print("\nZive įrašas:")
    file_name = get_filename(rec_dir, SubjCode)
    userNr, recNr = split_SubjCode(SubjCode)
    print(f"SubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2}")
    
    filepath = Path(rec_dir, str(SubjCode) + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    userId = data['userId']
    recordingId = data['recordingId']
    print(f"userId: {userId}  recordingId: {recordingId}")

    print("\nNuskaitome ir analizuojame EKG įrašo anotacijas")

     # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # SUFORMUOJAME EKG ĮRAŠUI TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])
    
    (unique, counts) = np.unique(atr_symbol, return_counts=True)
    total = counts.sum()
    print("test_symbols: ", unique, counts, total)

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)
    # print(f"df_rpeaks length: {len(df_rpeaks)} ")


# II-a dalis: nuskaitome ir analizuojame klasifikacijos rezultatus
    print("\nNuskaitome ir analizuojame klasifikacijos rezultatus")
    file_path = Path(res_dir, str(file_name))
    print(f"file_path: {file_path}")

    df_rpeaks = zive_read_df_data(file_path, 'automatic_classification')
    # print(f"df_rpeaks length: {len(df_rpeaks)} ")
    print(df_rpeaks.head())
    res_sample = df_rpeaks['sample'].to_numpy()
    res_symbol = df_rpeaks['annotation'].to_numpy()
    # print(res_symbol.dtype)

    print(f"\nres_sample: {res_sample[:40]}")
    print(f"\nres_symbol: {res_symbol[:40]}")
    print(f"\natr_sample: {atr_sample[:40]}")
    print(f"\natr_symbol: {atr_symbol[:40]}")
    

    pred_symbol = np.empty(len(atr_sample), dtype='object')
    # print(pred_symbol.dtype)

    for i, i_sample in enumerate(atr_sample):
        symbol = 'U'
        for j, j_sample in enumerate(res_sample):
            if (atr_sample[i] == res_sample[j]):
                symbol = res_symbol[j]
        pred_symbol[i] = symbol



    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    pred_labels = np.array([all_beats[symbol] for symbol in pred_symbol])

    (unique, counts) = np.unique(pred_symbol, return_counts=True)
    total = counts.sum()
    print("pred_symbols: ", unique, counts, total)

    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ", unique, counts, total)

    symbol_dict = dict(zip(all_beats.values(), all_beats.keys()))
    # print(symbol_dict)

     # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':symbol_dict[test_labels[i]], 'pred':symbol_dict[pred_labels[i]]})
            # classification.append({'sample':i_sample, 'annot':test_labels[i], 'pred':pred_labels[i]})
     # print(classification)

    # Vietų sąrašas išvedamas
    # classification = []  # Užblokuota
    if (classification):
        tot = 0
        tot_U = 0
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")
            if (row['pred'] != 'U'):
                tot += 1
            else:
                tot_U +=1     

    print(f"rpeaks total: {len(atr_sample)}")
    print(f"excluded rpeaks ('U'): {tot_U}")
    print(f"classified rpeaks: {len(atr_sample) - tot_U}")
    print(f"automaticRpeakAnnotations: {tot}")


Klasifikuojamų įrašų sąrašas: [10021]

Zive įrašas:
SubjCode: 10021 userNr: 1002 file_name: 1625402.027
userId: 60e1d80f93b55b41529e9eaa  recordingId: 60e1db5a93b55b56919e9ec0

Nuskaitome ir analizuojame EKG įrašo anotacijas
test_symbols:  ['N' 'S' 'V'] [748   4  13] 765
test_labels:  [0 1 2] [748   4  13] 765

Nuskaitome ir analizuojame klasifikacijos rezultatus
file_path: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/test_results/test/1625402.027
   sample annotation
0     188          U
1     346          N
2     507          N
3     667          N
4     828          N

res_sample: [ 188  346  507  667  828  985 1151 1323 1495 1679 1858 2041 2219 2392
 2557 2725 2895 3074 3253 3439 3618 3786 3950 4111 4275 4438 4605 4773
 4940 5111 5282 5449 5610 5770 5932 6095 6263 6433 6611 6802]

res_symbol: ['U' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N']

atr_sample: [ 188  346  507  667

In [ ]:
# PASIRUOŠIMAS

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
# warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas

               
print("\nAtliekama pūpsnių pacientų įrašuose klasifikacija")

# Pacientų įrašų sąrašas testavimui
file_path = 'testinis_sarasas.csv'
SubjCodes = [10021, 10022, 10083, 10091] #Testavimui, iš mokymo imties
# SubjCodes = [10021] #Testavimuas pacientų įrašų sąrašas

print("Klasifikuojamų įrašų sąrašas:", SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
# print(class_names)

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'recordingNr': int,
                                                             'sample': int, 'symbol': str, 'label': int})
all_beat_indices = all_beats_attr.index

index_start = 0
# Sukūriame masyvą, į kurį sudėsime visų įrašų pūpsnių anotuotus ir automatiškai surastus klasių numerius
validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

start_time = time.time()
# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    
    # Nuskaitome EKG įrašą (npy formatu)
    sign_raw = read_rec(rec_dir, SubjCode)
    signal_length = sign_raw.shape[0]
    signal = sign_raw

    # Surandame ir išvedame įrašo atributus
    print("\nZive įrašas:")
    file_name = get_filename(rec_dir, SubjCode)
    userNr, recNr = split_SubjCode(SubjCode)
    print(f"SubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2} signal_length: {signal_length}")
    
    filepath = Path(rec_dir, str(SubjCode) + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    userId = data['userId']
    recordingId = data['recordingId']
    print(f"userId: {userId}  recordingId: {recordingId}")

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # SUFORMUOJAME EKG ĮRAŠUI TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    (unique, counts) = np.unique(atr_symbol, return_counts=True)
    total = counts.sum()
    print("test_symbols: ", unique, counts, total)

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)
   
    pred_labels = predict_cnn_fda_vu_v1(signal, atr_sample, model_dir)
    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels, praleisti (šiuo atveju pirmas
    # ir paskutinis pūpsnys), o taip pat pakliuvęs į ommited sritį, pažymimi klase 3
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_labels ir pred_labels ilgiai")     

    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ",unique, counts, total)

    symbol_dict = dict(zip(all_beats.values(), all_beats.keys()))
    print(symbol_dict)
    # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':symbol_dict[test_labels[i]], 'pred':symbol_dict[pred_labels[i]]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    # classification = []  # Užblokuota
    if (classification):
        tot = 0
        tot_U = 0
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")
            if (row['pred'] != 'U'):
                tot += 1
            else:
                tot_U +=1     

    print(f"rpeaks total: {len(atr_sample)}")
    print(f"excluded rpeaks ('U'): {tot_U}")
    print(f"classified rpeaks: {len(atr_sample) - tot_U}")
    print(f"automaticRpeakAnnotations: {tot}")

    # SUFORMUOJAME FREIMĄ validation_set_stats SU PŪPSNIŲ KLASIŲ ANOTUOTAIS IR AUTOMATIŠKAI 
    # SURASTAIS KLASIŲ NUMERIAIS, IŠMETANT KLASES SU NUMERIU = 3

    # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recNr)]
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
    index_start = selected_ind[0]
    # print('\nSubjCode:',SubjCode, 'index_start:', index_start)   

    #  Praleisdami indeksą, jei masyvuose test_labels ir pred_labels yra reikšmė == 3,
    # suformuojame klasifikuotinų pūpsnių indeksų sąrašą
    for idx in range(len(atr_sample)):
        flag = (test_labels[idx] == 3) or (pred_labels[idx] == 3)
        if (flag == False):
            # Dėmesio: ?????????????????????????????????????????????
            # taisytina vieta, bus problemų su pandas 1.4.1
            validation_set_stats = validation_set_stats.append({'idx':index_start+idx, 
            'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

            # validation_set_stats = validation_set_stats.append({'idx':index_start+idx, 
            # 'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

    # Suformuojame klasių numerių msyvus confusion matricai skaičiuoti, surandama confusion matrica
    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 
    # print(all_beats_attr.info())
    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
   
    # Atsikračius pūpsnių su klase = 3 ir suformavus masyvus, pred_y turi būti tokio pat ilgio, kaip ir test_y
    if (len(test_y) != len(pred_y)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_y ir pred_y ilgiai")     

    # Skaičiuojame ir išvedame klasifikavimo lentelę
    confusion = confusion_matrix(test_y, pred_y)
    # print(confusion)
    pd.set_option('display.precision',3)
    show_confusion_matrix(confusion, class_names)

# print('\n')
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y, pred_y, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(sup[0]):>5} S:{(int(sup[1])):3} V:{int(sup[2]):3}" 
    str2 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str3 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str4 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    print(str1+str2+str3+str4)

    # print(len(validation_set_stats))
    # print(len(test_y))
    # print(len(pred_y))

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Sukūriame anotuotų ir automatiškai priskirtų klasių visų įrašų pūpsniams sąrašus 
validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')



Atliekama pūpsnių pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: [10021, 10022, 10083, 10091]
Sąrašas nuskaitytas iš: testinis_sarasas.csv


NameError: name 'read_rec' is not defined